# Movesのデータを分析する

## 1. 下準備

In [ ]:
# JupyterNotebookでデータ分析するときのの定番
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
%matplotlib inline

# 今回使用
import requests
import secret # INIファイルの読み書き 

In [ ]:
# アクセストークンをINIファイルから取得
access_token = secret.readAccessToken()

In [ ]:
# 取得したアクセストークンの中身
print(access_token)

## 2. ユーザープロフィールの取得

In [ ]:
api_url = 'https://api.moves-app.com/api/1.1/user/profile?access_token=' + access_token 
requests.get(api_url).json()

## 3. 2017年4月1ヶ月分のデータを取得



In [ ]:
api_url = 'https://api.moves-app.com/api/1.1/user/summary/daily/201704?access_token=' + access_token 

data = requests.get(api_url).json()
#requests.get(api_url).text

さて，これをどう料理するか

In [ ]:
type(data) # ドキュメントによると日々のデータのリストになっている

In [ ]:
type(data[0]) # 4/1のデータはdict型

4/1のデータだけ見る

In [ ]:
data[0]

In [ ]:
type(data[0]['summary'])

In [ ]:
type(data[0]['summary'][0])

各日付ごとにlist，その中に日付やサマリーがdictで入っていて，サマリーの値はアクティビティごとにlist,アクティビティの中はまたdict

In [ ]:
data[0]['summary'][0]['duration']

こうすればウォーキング，移動，バイクの3つのリストにデータを抽出できる

In [ ]:
walking = list()
transport = list()
motorcycle = list()
for d1 in data:
    date = d1["date"]
    for d2 in d1["summary"]:
        activity = d2["activity"]
        distance = d2["distance"]
        duration = d2["duration"]

        if activity=="walking":
            steps = d2["steps"]
            walking.append((date, activity, distance, steps))
        elif activity=="transport":
            transport.append((date, activity, distance, duration))
        elif activity=="motorcycle":
            motorcycle.append((date, activity, distance, duration))

In [ ]:
walking[0:5]

In [ ]:
transport[0:5]

In [ ]:
motorcycle[0:5]

あとでpandasのDataFrameに入れることを考えると，列ごとのリストにしたほうがよい

In [ ]:
date = list()
activity = list()
distance = list()
duration=list()
steps=list()

for d1 in data:
    for d2 in d1["summary"]:
        date.append(d1["date"])
        ac = d2["activity"]
        activity.append(ac)
        distance.append(d2["distance"])
        duration.append(d2["duration"])
        if ac=="walking":
            steps.append(d2["steps"])
        else:
            steps.append(np.nan)

        
df = DataFrame({'Date': date,
               'Activity': activity,
               'Distance': distance,
               'Duration': duration,
               'Steps': steps},
              columns=['Date','Activity', 'Distance', 'Duration', 'Steps'])

これでpandasのDataFrameに格納できる

In [ ]:
df.head(10)

## 4. 位置情報の取得

今度は位置情報を取得。4/15はバイクでツーリングに行った日。


In [ ]:
api_url2 = 'https://api.moves-app.com/api/1.1/user/storyline/daily/20170415?trackPoints=true&scope=activity&access_token=' + access_token

posdata = requests.get(api_url2).json()

In [ ]:
type(posdata[0]["segments"])

In [ ]:
type(posdata[0]["segments"][1])

In [ ]:
posdata[0]["segments"][1]["activities"][0]["trackPoints"][0]["lat"]

データを見てみると，位置情報があるのはactivityとplace

In [ ]:
lat=list()
lon=list()
time=list()
activity=list()
for d1 in posdata:
    for d2 in d1["segments"]:
        if "activities" in d2:
            for d3 in d2["activities"]:
                for d4 in d3["trackPoints"]:
                    activity.append(d3["activity"])
                    lat.append(d4["lat"])
                    lon.append(d4["lon"])
                    time.append(d4["time"])
        if "place" in d2:
            if "name" in d2["place"]:
                name = d2["place"]["name"]
            else:
                name = "Unknown"
            if name != "ホーム": # 自宅データは除外
                activity.append("Place:" + name)
                lat.append(d2["place"]["location"]["lat"])
                lon.append(d2["place"]["location"]["lon"])
                time.append(d2["endTime"])
                                                  
            
df2 = DataFrame({'Time': time,
               'Lat': lat,
               'Lon': lon,
               'Activity': activity},
              columns=['Time','Lat', 'Lon', 'Activity'])

In [ ]:
df2.head(10)

地図に表示するためにfoliumパッケージを使う。

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[35.01, 139], zoom_start=9)
for i, row in df2.iterrows():
    if row["Activity"][0:5]=="Place":
        folium.Marker([row["Lat"],row["Lon"]], popup=row["Activity"][6:]).add_to(m)
    else:
        folium.CircleMarker([row["Lat"],row["Lon"]], radius=2, color="#336699").add_to(m)
    #print([row["Lat"],row["Lon"]])
m